In [1]:
import path
with path.Path('..'):
    from services.theme_extractor.preprocessing import ArticlePreprocessJob, ArticlePreprocessor 

    from services.theme_extractor.wv_model import  WVModelBuilder

    from services.theme_extractor.clustering import Clusterer

    from services.theme_extractor.keyword_extraction import KeywordExtractor, KeywordExtractionJob, JointArticle

    from services.libs.data_model import ProcessedArticle, Theme, Article

In [2]:
kej = KeywordExtractionJob()

load_id = kej.get_latest_article_load().id

print(load_id)

articles, mapping = kej.get_articles_for_load_id(load_id)

467130b3-fe75-4f1f-b2fa-4757e15f11fd


In [3]:
from sqlalchemy import desc
session = kej.get_session()
raw_articles = session.query(Article).filter(Article.article_load_id == load_id).order_by(desc(Article.publish_date)).all()

In [4]:

raw_articles = raw_articles[:len(articles)]
all_tags = sum([art.source_tags for art in raw_articles] ,[])

In [5]:
from collections import Counter    

themes = Counter(all_tags)

In [6]:
theme_map = {}
for art in raw_articles:
    scores = [themes[tag] for tag in art.source_tags if themes[tag]]
    if len(scores) > 0:
        i = scores.index(min(scores))
        theme_map[art.id] = art.source_tags[i]
    else:
        theme_map[art.id] = 'unclassified'

In [8]:
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
import numpy as np

g_tags = [theme_map[art.id] for art in articles]

classified=  np.array(mapping) != -1




ami = adjusted_mutual_info_score(np.array(g_tags)[classified], np.array(mapping)[classified])
rand = adjusted_rand_score(np.array(g_tags)[classified], np.array(mapping)[classified])


50000
AMI: 0.35
Ramd: 0.35


In [9]:

print('AMI: {0:.2f}'.format(ami))
print('Ramd: {0:.2f}'.format(rand))

AMI: 0.35
Ramd: 0.07


In [10]:
for art in raw_articles[:10]:
    print(", ".join(art.source_tags))

world/world
sport/cycling,sport/lizzie-armitstead,sport/sport
law/us-supreme-court,law/us-constitution-and-civil-liberties,law/law-us,us-news/us-news,us-news/us-politics,us-news/donaldtrump,us-news/trump-administration,us-news/ruth-bader-ginsberg,us-news/us-elections-2020,us-news/republicans,us-news/democrats,us-news/us-senate,us-news/us-congress,world/world,us-news/healthcare,world/lgbt-rights,world/abortion,us-news/usdomesticpolicy
media/women-in-journalism,us-news/trump-administration,us-news/donaldtrump,lifeandstyle/women,us-news/us-news
media/charlie-hebdo,world/charlie-hebdo-attack,world/paris,world/france,world/europe-news,media/pressandpublishing,media/media,world/islam,world/world,world/religion
world/coronavirus-outbreak,technology/apps,politics/health,technology/technology,uk/uk,society/health,society/nhs,politics/pfi
sport/horse-racing,sport/sport,sport/horse-racing-tips
education/higher-education,uk/scotland,uk/glasgow,uk/uk,politics/nicola-sturgeon,politics/politics,educa